# Proceso de Conversion de los Datasets originales a los archivos CSV que serviran de datos para el juego

Los datasets elegidos para el desarrollo del juego son: 
* GRUPO A:
    - Lagos
    - Top 100 de temas musicales de Spotify 2010 a 2019
 * GRUPO B:
    - FIFA 2021 Complete Player Dataset

## Lagos

##### Proceso Paso a Paso

Primero la importacion del modulo os y la librería Pandas que seran necesarios para operar sobre los archivos

Tambien se importa desde el archivo config.py las rutas 
FILES_ROOT = ```os.path.abspath(os.path.join(ROOT, "files"))```
DATASET_ROOT = ```os.path.abspath(os.path.join(ROOT, "files", "datasets_final"))```

In [1]:
import os
import pandas as pd
from config import FILES_ROOT, DATASET_ROOT

### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ```def pasaje_sexagesimal(linea):```

Esta función es utilizada para convertir la columna de coordenadas en grados decimales, para lo cual hay que realizar una conversión desde el sistema sexagesimal a grados decimales.

Los grados de la coordenada quedan como parte entera, los minutos se dividen por 60, y los segundos se dividen por 3600, luego se suman obteniendo asi las coordenadas expresadas en grados decimales(con redondeo de 3 decimales).

Notar que todas las coordenadas estan expresadas en 2 posiciones (una para la latitud y otra para la longitud, por ende al string final de la coordenada se le agrega un "-" seguido de la letra inicial del punto cardinal al que hace referencia, es decir S O N E , siendo este el mismo que estaba en el dato original)

In [2]:
def pasaje_sexagesimal(linea):
    """ toma la coordenada por parametro, la "estandariza" reemplazando los simbolos de grados minutos y segundos
    por un caractger especial elegido.
    Luego hace un split sobre cada string utilizando dicho caracter especial y asi obteniendo en una lista los elementos 
    correspondientes a grados - minutos - segundos - punto cardinal en este orden, para finalmente realizar las conversiones
    casteando de int a str para las operaciones y volviendo a castear a str antes de retornar el valor completo procesado"""
    coordenadas = list(map(lambda coord: coord.replace('°', "-").replace("'","-").replace('"', "-"), linea.split(" ")))
    coordenadas = list(map(lambda elem : str(int(elem[0]) + round(int(elem[1])/60 + int(elem[2])/3600,3)) + "-" + elem[3] , map(lambda elem: elem.split('-'), coordenadas)))
    return " ".join(coordenadas)

#### ```def filtrado_lagos(data_frame):```

Recibe le data frame completo, se aplica la función que nos permite reordenar las columnas y también se convierte las coordenadas de grados a decimales.

- Se utiliza la función ```reindex``` provista por Pandas que nos permite reeordenar las columnas del data frame. En este caso se precisa ordenar para su posterior utilizacion por: "Ubicación”, “Superficie (km²)”, “Profundidad máxima (m)”, “Profundidad media (m)”, “Coordenadas” y “Nombre” . Además esta función descarta las columnas que no se ingresan por parámetro.

- Se le aplica la función  ```title``` a los encabezados de cada columna para que queden con el formato: Ubicación”, “Superficie (Km²)”, “Profundidad máxima (M)”, “Profundidad media (M)”, “Coordenadas” y “Nombre”.

- A la columna "Coordenadas" se le aplica la función ```apply``` (provista por Pandas), la cual le aplica una función a todos los elementos de una columna. A esta columna le vamos a aplicar la función ```pasaje_sexagesimal``` para asi guardar luego del mappeo las coordenadas ya convertidas a grados decimales.

- Finalmente se retorna el data frame con los ```datos``` de los lagos ya ordenados y con sus coordenadas convertidas.


In [3]:
def filtrado_lagos(data_frame):
   """Se recibe el data frame completo. 
      - Se reordenan las columnas por : 'Ubicación', 'Superficie (km²)', 'Profundidad máxima (m)', 'Profundidad media (m)', 'Coordenadas' y 'Nombre'
      - Se ponen en mayúscula la primer letra de cada palabra de los nombres de las columnas.
      - A los datos de las columnas 'Profundidad Máxima (M)' y 'Profundidad Media (M)', que sean nulos (NaN), se las reemplazan por el String 'Desconocida'
      - Se convierten las coordenadas en Grados, Minutos y Segundos a coordenadas decimales
      - Se retorna el data frame
      """

   data_frame = data_frame.reindex(columns=['Ubicación', 'Superficie (km²)', 'Profundidad máxima (m)', 'Profundidad media (m)', 'Coordenadas', 'Nombre'])
   data_frame.columns = data_frame.columns.str.title()

   data_frame['Profundidad Máxima (M)'] = data_frame['Profundidad Máxima (M)'].fillna("Desconocida")
   data_frame['Profundidad Media (M)'] = data_frame['Profundidad Media (M)'].fillna("Desconocida")
   data_frame['Coordenadas'] = data_frame['Coordenadas'].apply(pasaje_sexagesimal)
   return data_frame
   

#### ```def crear_lagos(data_frame):```
Recibe el data frame completo (cada línea interna es un lago y cada campo dentro de esta lista es un dato correspondiente a dicho lago).

- Primero: se llama la función ```filtrado_lagos(data_frame```).
- Se retorna el data frame modificado

In [4]:
def crear_lagos(data_frame):
    """Recibe el data frame completo, llama a la función 'filtrado_lago()', enviando el data frame como parámetro y retorna el data frame"""
    data_frame = filtrado_lagos(data_frame)
    return data_frame


## 2. Declaracion de las rutas a utilizar dentro de la estructura de archivos del juego

La estructura cuenta con una carpeta llamada "files".

En dicha carpeta, se encuentra una carpeta con el nombre "datasets_base" en la cual se almacenan los datasets originales.

Los datasets modificados y listos para la utilizacion de acuerdo a los requerimientos del juego, seran guardados dentro de la carpeta "files" en una carpeta llamada "datasets_final"

Es decir, desde donde se ejecuta el programa:

. >files>datasets_base                      -   aqui se ecuentran los archivos csv originales

. >files>datasets_final                     -   aqui se almacenaran los archivos csv procesados y posteriormente utilizados para la funcionalidad del juego

In [5]:
origen = "datasets_base"

## 3. Apertura del archivo ORIGINAL y su procesamiento

Se lee el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Esto lo realizamos usando la función de ```read_csv``` que nos proveé Pandas, indicando por parámetros que vamos a utilizar el caracter "," como separador. Esto nos genera un data frame que lo guardaremos en una variable para ser manipulado.


In [6]:
data_frame = pd.read_csv(os.path.join(FILES_ROOT, origen, 'lagos.csv'), sep=',', encoding="utf-8")


## 4. Creacion del archivo FINAL

Una vez guardado el data frame en una variable llamamos a la función ```crear_lagos(data_frame)```, enviando el data frame como parámetro.

Con el data frame ya modificado, lo guardamos en un nuevo CSV para su utilización.

In [7]:
data_frame = crear_lagos(data_frame)
data_frame.to_csv(os.path.join(DATASET_ROOT, 'lagos_filtrado_pandas.csv'), encoding="utf-8")


## Top 100 de temas musicales de Spotify 2010 a 2019

##### Proceso Paso a Paso

La importacion de los modulos os y Pandas que seran necesarios para operar sobre los archivos ya fue realizada anteriormente por lo que no es necesario hacerlo nuevamente, ya que se realiza una sola vez por sesion de interprete.


### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ``` def filtrado_spotify(data_frame):```

Esta función recibe el data frame completo, y les aplica:

- Primero: Se utiliza la función ```reindex``` provista por Pandas que nos permite reeordenar las columnas del data frame. En este caso se precisa ordenar para su posterior utilizacion por: "top genre", "artist type", "year released", "top year", "bpm" y "artist" . Además esta función descarta las columnas que no se ingresan por parámetro.

- Segundo: Se le aplica la función  ```title``` a los encabezados de cada columna para que queden con el formato: "Top Genre, "Artist Type", "Year Released", "Top Year", "Bpm" y "Artist".

- Tercero: Se utiliza la función rename para renombrar el encabezado de la columna "Bpm", para que quede con el formato "BPM"

- Cuarto: Se retorna el data frame

In [8]:
def filtrado_spotify(data_frame):
    '''Se recibe data frame completo. Y:
        - Se reordenan las columnas por 'top genre', 'artist type', 'year released', 'top´year', 'bpm', 'artist'.
        - Se ponen en mayúscula la primer letra de cada palabra de los nombres de las columnas.
        - Se renombra la columna 'Bpm' por su equivalente en mayúsculas
        - Se retorna el data frame modificado.
    '''
    
    data_frame = data_frame.reindex(columns=['top genre', 'artist type', 'year released', 'top year', 'bpm', 'artist'])
    data_frame.columns = data_frame.columns.str.title()
    data_frame.rename(columns={'Bpm':'BPM'}, inplace=True)
    return data_frame


#### ```def estilo_generos(genero):```

Funcion auxiliar que evalua si el Genero pasado por parametro - String - contiene alguna de las excepciones de estilo para los nombres de los generos que se encuentran en la lista de excepciones.

- Primero: Se crea la lista de excepciones. En este caso son: "edm", "dfw", "uk", "lgbtq+" y "r&b"

- Segundo: Si el genero está compuesto por más de una palabra, se separa en una lista que contiene cada una de las palabras del genero para su fácil manipulación.

- Tercero: Se crea una lista por comprensión. En la cual, si la palabra actual de la lista separada se encuentra en la lista de excepciones, se le aplica la función ```upper``` para convertir a mayúsculas todas las letras de la palabra. De caso contrario, se aplica la función ```title``` para poner en mayúscula la primera letra de la palabra. En cualquiera de los dos casos se guarda en la lista por comprensión.

- Cuarto: Se retorna la lista haciendo un ```join``` de la lista por comprensión, para generar nuevamente el genero en un único String.

In [9]:
def estilo_generos(genero):
    """Se recibe una línea de la columna 'genero' del data frama. Para convertir las palabras 
    - Se crea una lista con las excepciones de las palabras a modificar. En este caso 'edm', 'dfw', 'uk', 'lgbtq+' y 'r&b'
    - En caso de que haya más de una palabra en el genero musical, se las separa en una lista
    - Se crea una lista por comprensión, en el que se itera por la lista de palabras separadas, y por cada palabra pregunta si la palabra se encuentra en la lista de excepsiones. En caso de encontrarse, se la pone la palabra entera en mayúsculas. En caso de no encontrarse, se pone en mayúscula la primer letra de la palabra
    - Con la lista de palabras modificadas, se retorna un String compuesto por cada palabra de la lista unidas por un espacio
    """

    excepciones=['edm', 'dfw', 'uk', 'lgbtq+', 'r&b']
    genero = genero.split(' ')
    lista_genero = [palabra.upper() if palabra in excepciones else palabra.title() for palabra in genero]
    return ' '.join(lista_genero)


#### ```def crear_spotify(data_frame):```
Recibe el data frame completo para su modificación.

- Primero: Se aplica al data frame completo la función ```dropna(how='all')```. Esto descarta las filas en las que todos sus campos no tengan datos (NaN).

- Segundo: Se llama a la función ```convert_dtypes(convert_integer=True)```, para convertir los valores float a integer.

- Tercero: Se invoca la función ```filtrado_spotify(data_frame)```, enviando el data frame para su modificación.

- Cuarto: Se le aplica a la columna "Top Genre", mediante ```apply```, la función ```estilo_generos()```

- Quinto: Se retorna el data frame

In [10]:
def crear_spotify(data_frame):
    """ Se recibe el data frame completo.
        - Se eliminan las filas en las que todas sus columnas no tengan datos (NaN)
        - Se convierten todos los valores numéricos a enteros
        - Se reordenan las columnas, se renombra la columna 'bpm' y se pone en mayúscula la primer letra de cada nombre de columna
        - Se ponen en mayúsculas las palabras 'edm', 'dfw', 'uk', 'lgbtq+' y 'r&b' de cada genero musical 
        - Se retorna el data frame
    """

    data_frame = data_frame.dropna(how='all')
    data_frame = data_frame.convert_dtypes(convert_integer=True)
    data_frame = filtrado_spotify(data_frame)
    data_frame['Top Genre'] = data_frame['Top Genre'].apply(estilo_generos)
    return data_frame

## 3. Apertura del archivo ORIGINAL y su procesamiento

Se lee el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Esto lo realizamos usando la función de ```read_csv``` que nos proveé Pandas, indicando por parámetros que vamos a utilizar el caracter "," como separador. Esto nos genera un data frame que lo guardaremos en una variable para ser manipulado.


In [11]:
ruta = os.path.join(FILES_ROOT, origen)
data_frame = pd.read_csv(os.path.join(ruta, 'spotify.csv'), sep=',', encoding="utf-8")

## 4. Creacion del archivo FINAL

Una vez guardado el data frame en una variable llamamos a la función ```crear_spotify(data_frame)```, enviando el data frame como parámetro.

Con el data frame ya modificado, lo guardamos en un nuevo CSV para su utilización.

In [12]:

data_frame = crear_spotify(data_frame)
data_frame.to_csv(os.path.join(DATASET_ROOT, 'spotify_filtrado_pandas.csv'), encoding="utf-8")

## FIFA 2021 Complete Player Dataset

##### Proceso Paso a Paso

La importacion de los modulos os y Pandas que seran necesarios para operar sobre los archivos ya fue realizada anteriormente por lo que no es necesario hacerlo nuevamente, ya que se realiza una sola vez por sesion de interprete.


### 1. Definir las funciones que serviran para el procesamiento de los datos

#### ``` def potencial(score):```

Recibe por parametro el String score correspondiente al valor de la columna del data frame.

Lo convierte a integer para luego hacer las comparaciones y asignarle de acuerdo al valor en la escala, un adjetivo calificativo que luego reemplazara al valor numerico en la columna de potencial del data frame.

In [13]:
def potencial(score):
    """ Recibe por parametro el String score correspondiente al valor de la columna del data frame. Lo convierte a integer para luego hacer las comparaciones y asignarle de acuerdo
    al valor en la escala, un adjetivo calificativo que luego reemplazara al valor numerico en la columna de potencial del data frame final. Se retorna dicho valor expresado como 
    adjetivo calificativo. """

    score = int(score)
    if  score < 60: 
        score = "Regular" 
    elif score <= 79:
        score = "Bueno"
    elif score <= 89:
        score = "Muy Bueno"
    else:
        score = "Sobresaliente"
    return score


#### ``` def traducir_posiciones(posicion):```

Recibe por parametro el String con la posicion o posiciones del jugador de futbol.

1. Se crea un diccionario con las traducciones de cada sigla en inglés a su traducción en español.
2. Lo divide mediante el caracter " | " que separa las multiples posiciones del jugador (en caso de que tenga más de una posición), obteniendo asi una lista de posiciones
3. Para cada posible posicion del jugador, segun sean las siglas de la misma, se reemplaza su valor por la descripcion en español de la misma
4. Vuelve a unir mediante el caracter especial " | " las posiciones ya traducidas y retorna el dicho String


In [14]:

def traducir_posiciones(posicion):
    '''Se recibe una línea de la columna de posiciones del data frame.
    - Se crea un diccionario en el que cada clave son las siglas de una posición en inglés, y su valor es su traducción en español
    - En caso de que haya más de una posición por línea, se las separa en una lista
    - Se crea una lista por comprensión, en el que se itera por cada posición en la lista de posiciones separadas. Y a cada posición se la reemplaza por su valor en el diccionario de traducciones
    - Se devuelve un String compuesto por las cada posición traducida, unida por el caracter "|" '''
    posiciones={"GK":"Arquero", 
            "ST":"Segundo Delantero",
            "CF":"Delantero Centro", 
            "RW":"Extremo Derecho", 
            "LW":"Extremo Izquierdo", 
            "LS":"Delantero Izquierdo", 
            "RS":"Delantero Derecho", 
            "CAM":"Centrocampista Ofensivo Central", 
            "RAM":"Centrocampista Ofensivo Derecho", 
            "LAM": "Centrocampista Ofensivo Izquierdo", 
            "LM":"Mediocampista Izquierdo", 
            "LCM":"Mediocampista Central Izquierdo", 
            "CM":"Mediocampista Central", 
            "RCM":"Mediocampista Central Derecho", 
            "RM":"Mediocampista Derecho", 
            "LDM": "Mediocampista Defensivo Izquierdo", 
            "CDM":"Mediocampista Defensivo Central", 
            "RDM":"Mediocampista Defensivo Derecho", 
            "RWB":"Defensor Extremo Derecho", 
            "RB":"Defensor Lateral Derecho", 
            "RCB":"Defensor Central Derecho", 
            "CB":"Defensor Central", 
            "LCB":"Defensor Central Izquierdo", 
            "LB":"Defensor Lateral Izquierdo", 
            "LWB":"Defensor Extremo Izquierdo", 
            "SW":"Libero"}

    lista_posiciones = posicion.split('|')
    lista_remplazada = [posiciones[posicionActual] for posicionActual in lista_posiciones]
    return('|'.join(lista_remplazada))

#### ``` def filtrado_fifa(data_frame):```

Recibe por parametro el data frame. 

- Se utiliza la función ```reindex``` provista por Pandas que nos permite reeordenar las columnas del data frame. En este caso se precisa ordenar por: "team", "nationality", "position", "age", "potential", "name". Además esta función descarta las columnas que no se ingresan por parámetro.

- Se le aplica la función  ```title``` a los encabezados de cada columna para que queden con el formato: “Team”, “Nationality”, “Position”, “Age”, “Potential” y “Name”.

- Por último se le aplica la función ```strip``` a los datos de la columna "Team", para eliminar los espacios al final y al principio de del String.

- Se retorna el data frame con las columnas reeordenadas y el formato correcto de cada encabezado.


In [15]:
def filtrado_fifa(data_frame):
    """ Recibe el data frame completo. Y:
    - Se reordenan las columas por: 'team', 'nationality', 'position', 'age', 'potential' y 'name'
    - Se ponen en mayúscula la primer letra de cada palabra de los nombres de las columnas.
    - Se eliminan los espacios delante y detrás de los nombres de los equipos
    - Se retorna el data frame modificado"""
    
    data_frame = data_frame.reindex(columns=['team', 'nationality', 'position', 'age', 'potential', 'name'])
    data_frame.columns = data_frame.columns.str.title()
    data_frame['Team'] = data_frame['Team'].str.strip()
    return data_frame

#### ```def crear_fifa(data_frame):```
Recibe el data frame completo (cada línea interna es un jugador y cada campo dentro de esta lista es un dato correspondiente a dicho jugador).

- Primero: se llama a la función ```filtrado_fifa```, para darle el formato deseado, reeordenar las columnas al orden que nos sea más conveniente y descartar las innecesarias.

- Segundo: Se le aplica a cada dato de la columna "Potential", mediante el método ```apply```, la función ```potencial(score)```. Esto lo hacemos convertir el potencial en formato numérico de cada jugador a su equivalente con palabras.

- Tercero: Nuevamente usando ```apply```, le aplicamos la función ```traducir_posiciones(posicion)``` a cada dato de la columna "Posicion". Para traducir su las posiciones con siglas en inglés al español.

In [16]:
def crear_fifa(data_frame):
    """ Se recibe el data frame completo:
        - Se llama a la función 'filtrado_fifa' pasando el data frame como parametro
        - Se modifican los potenciales numéricos de los jugadores, por su equivalente en palabras
        - Se modifican las posiciones en inglés por su traducción en español
        - Se retorna el data frame
    """

    data_frame = filtrado_fifa(data_frame)
    data_frame['Potential'] = data_frame['Potential'].apply(potencial)
    data_frame['Position'] = data_frame['Position'].apply(traducir_posiciones)
    return data_frame

## 3. Apertura del archivo ORIGINAL y su procesamiento

Se lee el archivo origen (con el csv sin procesar aun), haciendo un Join de la "ruta_completa" con "origen".

Esto lo realizamos usando la función de ```read_csv``` que nos proveé Pandas, indicando por parámetros que vamos a utilizar el caracter ";" como separador. Esto nos genera un data frame que lo guardaremos en una variable para ser manipulado.

In [17]:
data_frame = pd.read_csv(os.path.join(FILES_ROOT, origen, 'FIFA-21.csv'), sep=';', encoding="utf-8")

## 4. Creacion del archivo FINAL

Una vez guardado el data frame en una variable llamamos a la función ```crear_fifa(data_frame)```, enviando el data frame como parámetro.

Con el data frame ya modificado, lo guardamos en un nuevo CSV para su utilización.

In [18]:
data_frame = crear_fifa(data_frame)
data_frame.to_csv(os.path.join(DATASET_ROOT,'fifa_filtrado_pandas.csv'), encoding="utf-8")